In [1]:
import sqlite3
import pandas as pd
conn = sqlite3.connect('chinook.db')

# Selecionar através de SQL os 3 gêneros mais vendidos no Brasil

In [2]:
query = """
SELECT GR.Name, Sum(II.Quantity) as 'Qtd de Vendas no Brasil'
FROM invoice_items II
JOIN invoices IV
ON II.InvoiceId = IV.InvoiceId
JOIN tracks TR
ON II.TrackId = TR.TrackId
JOIN genres GR
ON TR.GenreId = GR.GenreId
WHERE IV.BillingCountry = 'Brazil'
GROUP BY GR.Name
ORDER BY Sum(II.Quantity) DESC
LIMIT 3;
"""
df = pd.read_sql(query, conn)
print(df)

    Name  Qtd de Vendas no Brasil
0   Rock                       81
1  Latin                       53
2  Metal                       15


# Selecionar através de SQL o total de vendas (quantidade e valor total) de cada faixa do álbum 'Mais Do Mesmo'

In [3]:
query = """
SELECT TR.Name, Sum(II.Quantity) as 'Qtd Vendas', Sum(II.UnitPrice * II.Quantity) as 'Valor Total'
FROM invoice_items II
JOIN tracks TR
ON II.TrackId = TR.TrackId
JOIN albums AL
ON TR.AlbumId = AL.AlbumId
WHERE title = 'Mais Do Mesmo'
GROUP BY TR.Name
ORDER BY 'Valor Total' DESC;
"""
df = pd.read_sql(query, conn)
print(df)

                Name  Qtd Vendas  Valor Total
0      Tempo Perdido           2         1.98
1               Será           1         0.99
2      Pais E Filhos           1         0.99
3  Meninos E Meninas           1         0.99
4             Indios           1         0.99
5                Giz           1         0.99
6          Dezesseis           1         0.99


# Selecionar através de SQL o valor total de venda por nome completo do vendedor(a) em 2012

In [4]:
query = """
SELECT CT.FirstName || ' ' || CT.LastName as 'Nome Completo do Vendedor',
Sum(II.Quantity) as 'Qtd Vendas',
CAST(Round(Sum(II.UnitPrice * II.Quantity),2) AS DECIMAL) as 'Valor Total'
FROM invoice_items II
JOIN invoices IV
ON II.InvoiceId = IV.InvoiceId
JOIN customers CT
ON IV.CustomerId = CT.CustomerId
WHERE strftime('%Y', InvoiceDate) = '2012'
GROUP BY CT.FirstName, CT.LastName
ORDER BY CAST(Round(Sum(II.UnitPrice * II.Quantity),2) AS DECIMAL) DESC;
"""
df = pd.read_sql(query, conn)
print(df)

   Nome Completo do Vendedor  Qtd Vendas  Valor Total
0         Richard Cunningham          16        25.84
1             João Fernandes          23        24.77
2             Fernanda Ramos          25        24.75
3           Joakim Johansson          25        24.75
4                Mark Taylor          23        22.77
5      František Wichterlová          16        18.84
6           Isabelle Mercier          16        18.84
7              Julia Barnett          12        17.88
8            Heather Leacock          16        17.84
9              Frank Ralston          12        15.88
10            Edward Francis          16        15.84
11              Kara Nielsen          16        15.84
12             Lucas Mancini          16        15.84
13            Luís Gonçalves          16        15.84
14           Alexandre Rocha          12        11.88
15             Astrid Gruber          12        11.88
16           Ladislav Kovács          12        11.88
17          Stanisław Wójcik

# Listar em uma única query o total de músicas, total de músicas com o tipo de mídia MPEG e o total com tipo de mídia AAC para cada tipo de gênero. Exemplo de saída: 

In [5]:
query = """
SELECT GR.Name,
Count(TR.TrackId) as 'Total de Músicas',
SUM(CASE WHEN MT.Name like '%MPEG%' THEN 1 ELSE 0 END) AS 'Total mídia MPEG',
SUM(CASE WHEN MT.Name like '%AAC%' THEN 1 ELSE 0 END) AS 'Total mídia AAC'
FROM tracks TR
JOIN genres GR
ON TR.GenreId = GR.GenreID
JOIN media_types MT
ON TR.MediaTypeId = MT.MediaTypeId
GROUP BY GR.Name
ORDER BY Count(TR.TrackId) DESC;
"""
df = pd.read_sql(query, conn)
print(df)

                  Name  Total de Músicas  Total mídia MPEG  Total mídia AAC
0                 Rock              1297              1211               86
1                Latin               579               578                1
2                Metal               374               374                0
3   Alternative & Punk               332               332                0
4                 Jazz               130               127                3
5             TV Shows                93                93                0
6                Blues                81                81                0
7            Classical                74                 0               74
8                Drama                64                64                0
9             R&B/Soul                61                49               12
10              Reggae                58                58                0
11                 Pop                48                14               34
12          